**Generation of Drug Fingerprints using Adversarial Autoencoder**

Initialise packages and variables

In [0]:
import subprocess
import numpy as np
import tensorflow as tf

from tqdm import tqdm
from google.colab import files

In [0]:
# All global variables and hyperparameters

train = 'train_aae.txt'
test = 'test_aae.txt'

num_of_epochs = 100
batch_size_for_model = 1024
num_of_test_instances = 85399

input_layer_size, latent_layer_size, output_layer_size = 167, 20, 167
middle_layer_sizes = [256, 256]

discriminator_sizes = [64, 64, 8, 1]

logdir = 'tmp/aae/1/'

Data selection procedures

In [0]:
# Data selection and processing

def batch_gen(data, batch_n):
    inds = list(range(data.shape[0]))
    np.random.shuffle(inds)
   
    for i in range(int(data.shape[0] / batch_n)):
        ii = inds[i*batch_n:(i+1)*batch_n]
        yield data[ii, :]
        
def buffered_gen(f, batch_n=1024, buffer_size=2000):
    inp = open(f)
    data = []
    
    for i, line in enumerate(inp):
        data.append(np.array(list(map(float, line.strip().split('\t')[1]))))
        
        if (i+1) % (buffer_size * batch_n) == 0:
            bgen = batch_gen(np.vstack(data), batch_n)
            for batch in bgen:
                yield batch
            data = []
            
    else:
        bgen = batch_gen(np.vstack(data[:-1]), batch_n)
        
        for batch in bgen:
            yield batch
            
def load_test():
    with open(test) as inp:
        data = [np.array(list(map(float, line.strip().split('\t')[1]))) for line in inp]
    return np.vstack(data)

Definitions of AAE model and utility functions

In [0]:
# Utility functions for AAE model

def he_initializer(size):
    return tf.random_normal_initializer(mean=0.0, stddev=np.sqrt(1. / size), seed=None, dtype=tf.float32)
  
def linear_layer(tensor, input_size, out_size, init_fn=he_initializer,):
    W = tf.get_variable('W', shape=[input_size, out_size], initializer=init_fn(input_size))
    b = tf.get_variable('b', shape=[out_size], initializer=tf.constant_initializer(0.1))
    return tf.add(tf.matmul(tensor, W), b)
  
def bn_layer(tensor, size, epsilon=0.0001):
    batch_mean, batch_var = tf.nn.moments(tensor, [0])
    scale = tf.get_variable('scale', shape=[size], initializer=tf.constant_initializer(1.))
    beta = tf.get_variable('beta', shape=[size], initializer=tf.constant_initializer(0.))
    return tf.nn.batch_normalization(tensor, batch_mean, batch_var, beta, scale, epsilon)
  
def sample_prior(loc=0., scale=1., size=(64, 10)):
    return np.random.normal(loc=loc, scale=scale, size=size)


In [0]:
# Class definition for AAE

class AAE(object):
    def __init__(self,
                 gpu_config=None,
                 batch_size=batch_size_for_model, 
                 input_space=input_layer_size,
                 latent_space=latent_layer_size,
                 middle_layers=middle_layer_sizes,
                 disc_layers=discriminator_sizes,
                 activation_fn=tf.nn.tanh,
                 learning_rate=0.001,
                 initializer=he_initializer):

        self.batch_size = batch_size
        self.input_space = input_space
        self.latent_space = latent_space
        if middle_layers is None:
            self.middle_layers = [256, 256]
        else:
            self.middle_layers = middle_layers
        self.activation_fn = activation_fn
        self.learning_rate = learning_rate

        self.initializer = initializer

        tf.reset_default_graph()
        
        self.input_x = tf.placeholder(tf.float32, [None, input_space])
        self.z_tensor = tf.placeholder(tf.float32, [None, latent_space])

        # Encoder net: 167->256->256->10
        with tf.variable_scope("encoder"):
            self.encoder_layers = self.encoder()
            self.encoded = self.encoder_layers[-1]
        
        # Decoder net: 20->256->256->167
        with tf.variable_scope("decoder"):
            self.decoder_layers = self.decoder(self.encoded)
            self.decoded = self.decoder_layers[-1]
            tf.get_variable_scope().reuse_variables()
            self.generator_layers = self.decoder(self.z_tensor)
            self.generated = tf.nn.sigmoid(self.generator_layers[-1])

        # Discriminator net: 20->64->64->8->1
        sizes = disc_layers
        with tf.variable_scope("discriminator"):
            self.disc_layers_neg = self.discriminator(self.encoded, sizes)
            self.disc_neg = self.disc_layers_neg[-1]
            tf.get_variable_scope().reuse_variables()
            self.disc_layers_pos = self.discriminator(self.z_tensor, sizes)
            self.disc_pos = self.disc_layers_pos[-1]

        self.pos_loss = tf.nn.relu(self.disc_pos) - self.disc_pos + tf.log(1.0 + tf.exp(-tf.abs(self.disc_pos)))
        self.neg_loss = tf.nn.relu(self.disc_neg) + tf.log(1.0 + tf.exp(-tf.abs(self.disc_neg)))
        self.disc_loss = tf.reduce_mean(tf.add(self.pos_loss, self.neg_loss))
        
        tf.summary.scalar("discriminator_loss", self.disc_loss)
            
        self.enc_loss = tf.reduce_mean(tf.nn.relu(self.disc_neg) - self.disc_neg + tf.log(1.0 + tf.exp(-tf.abs(self.disc_neg))))
        batch_logloss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.decoded, labels=self.input_x), 1)
        self.dec_loss = tf.reduce_mean(batch_logloss)
        
        tf.summary.scalar("encoder_loss", self.enc_loss)
        tf.summary.scalar("decoder_loss", self.dec_loss)
        
        disc_ws = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='discriminator')
        enc_ws = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='encoder')
        ae_ws = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='encoder') + \
                tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='decoder')
            
        self.train_discriminator = tf.train.AdamOptimizer(self.learning_rate).minimize(self.disc_loss, var_list=disc_ws)
        self.train_encoder = tf.train.AdamOptimizer(self.learning_rate).minimize(self.enc_loss, var_list=enc_ws)
        self.train_autoencoder = tf.train.AdamOptimizer(self.learning_rate).minimize(self.dec_loss, var_list=ae_ws)

        if gpu_config is None:
            gpu_config = tf.ConfigProto()
            gpu_config.gpu_options.per_process_gpu_memory_fraction = 0.4
        
        self.sess = tf.Session(config=gpu_config)
        self.init_net()
        
    def encoder(self):
        sizes = self.middle_layers + [self.latent_space]
        with tf.variable_scope("layer-0"):
            encoder_layers = [linear_layer(self.input_x, self.input_space, sizes[0])]
        for i in range(len(sizes) - 1):
            with tf.variable_scope("layer-%i" % (i+1)):
                activated = self.activation_fn(encoder_layers[-1])
#                 normed = bn_layer(activated, sizes[i])
                next_layer = linear_layer(activated, sizes[i], sizes[i+1])
            encoder_layers.append(next_layer)
            
        return encoder_layers

    def decoder(self, tensor):
        sizes = self.middle_layers[::-1] + [self.input_space]
        with tf.variable_scope("layer-0"):
            decoder_layers = [linear_layer(tensor, self.latent_space, sizes[0])]
        for i in range(len(sizes) - 1):
            with tf.variable_scope("layer-%i" % (i+1)):
                activated = self.activation_fn(decoder_layers[-1])
#                 normed = bn_layer(activated, sizes[i])
                next_layer = linear_layer(activated, sizes[i], sizes[i+1])
            decoder_layers.append(next_layer)
        
        return decoder_layers
    
    def discriminator(self, tensor, sizes):
        with tf.variable_scope("layer-0"):
            disc_layers = [linear_layer(tensor, self.latent_space, sizes[0])]
        for i in range(len(sizes) - 1):
            with tf.variable_scope("layer-%i" % (i+1)):
                activated = tf.nn.tanh(disc_layers[-1])
#                 normed = bn_layer(activated, sizes[i])
                next_layer = linear_layer(activated, sizes[i], sizes[i+1])
            disc_layers.append(next_layer)

        return disc_layers
    
    def init_net(self):
        init = tf.global_variables_initializer()
        self.sess.run(init)        
    
    def train(self, log, epochs):
        sess = self.sess
        saver = tf.train.Saver()
        hist = []
        test_data = load_test()
        
        merged_summary = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('tmp/aae/1')
        train_writer.add_graph(sess.graph)
        
        for e in tqdm(range(epochs)):
            print (log, "epoch #%d" % (e+1))
            log.flush()
            train_gen = buffered_gen(train, batch_n=self.batch_size)
            for i, batch_x in enumerate(train_gen):
                if i%3 == 0:
                    batch_z = sample_prior(scale=1.0, size=(len(batch_x), self.latent_space))
                    sess.run(self.train_discriminator, feed_dict={self.input_x: batch_x, self.z_tensor: batch_z})
                elif i%3 == 1:
                    sess.run(self.train_encoder, feed_dict={self.input_x: batch_x})
                else:
                    sess.run(self.train_autoencoder, feed_dict={self.input_x: batch_x})
                if i % 5 == 0:
                    batch_z = sample_prior(scale=1.0, size=(len(test_data), self.latent_space))
                    s = sess.run(merged_summary, feed_dict={self.input_x: test_data, self.z_tensor: batch_z})
                    train_writer.add_summary(s, i)
                if i%10000 == 0:
                    batch_z = sample_prior(scale=1.0, size=(len(test_data), self.latent_space))
                    
                    losses = sess.run([self.disc_loss, self.enc_loss, self.dec_loss],
                                      feed_dict={self.input_x: test_data, self.z_tensor: batch_z})
                    discriminator_loss, encoder_loss, decoder_loss = losses
                    print (log, "disc: %f, encoder : %f, decoder : %f" % (discriminator_loss/2., encoder_loss, decoder_loss))
                    
                    log.flush()
            else:         
                saver.save(sess, './fpt.aae.%de.model.ckpt' % e)
                batch_z = sample_prior(scale=1.0, size=(len(test_data), self.latent_space))
                losses = sess.run([self.disc_loss, self.enc_loss, self.dec_loss],
                                  feed_dict={self.input_x: test_data, self.z_tensor: batch_z})


                discriminator_loss, encoder_loss, decoder_loss = losses
                print (log, "disc: %f, encoder : %f, decoder : %f" % (discriminator_loss/2., encoder_loss, decoder_loss))
                    
                log.flush()
                hist.append(decoder_loss)
        return hist
    
    def load(self, model):
        saver = tf.train.Saver()
        saver.restore(self.sess, model)

Training of model

In [0]:
# Initialise AAE

aae = AAE(batch_size=1024)

In [7]:
# Train AAE

with open('./fpt.aae.log', 'w') as log:
     hist1 = aae.train(log, num_of_epochs)

  0%|          | 0/100 [00:00<?, ?it/s]

<_io.TextIOWrapper name='./fpt.aae.log' mode='w' encoding='UTF-8'> epoch #1
<_io.TextIOWrapper name='./fpt.aae.log' mode='w' encoding='UTF-8'> disc: 0.610797, encoder : 0.824400, decoder : 121.533417


  1%|          | 1/100 [00:04<08:06,  4.92s/it]

<_io.TextIOWrapper name='./fpt.aae.log' mode='w' encoding='UTF-8'> disc: 0.757299, encoder : 0.602593, decoder : 77.857780
<_io.TextIOWrapper name='./fpt.aae.log' mode='w' encoding='UTF-8'> epoch #2
<_io.TextIOWrapper name='./fpt.aae.log' mode='w' encoding='UTF-8'> disc: 0.680611, encoder : 0.744297, decoder : 77.857780


  2%|▏         | 2/100 [00:09<07:47,  4.77s/it]

...

<_io.TextIOWrapper name='./fpt.aae.log' mode='w' encoding='UTF-8'> disc: 0.227922, encoder : 1.440739, decoder : 59.231255
<_io.TextIOWrapper name='./fpt.aae.log' mode='w' encoding='UTF-8'> epoch #99
<_io.TextIOWrapper name='./fpt.aae.log' mode='w' encoding='UTF-8'> disc: 0.210966, encoder : 1.548348, decoder : 59.231255


 99%|█████████▉| 99/100 [07:24<00:04,  4.49s/it]

<_io.TextIOWrapper name='./fpt.aae.log' mode='w' encoding='UTF-8'> disc: 0.251626, encoder : 1.246556, decoder : 57.076916
<_io.TextIOWrapper name='./fpt.aae.log' mode='w' encoding='UTF-8'> epoch #100
<_io.TextIOWrapper name='./fpt.aae.log' mode='w' encoding='UTF-8'> disc: 0.231803, encoder : 1.347641, decoder : 57.076916


100%|██████████| 100/100 [07:28<00:00,  4.49s/it]

<_io.TextIOWrapper name='./fpt.aae.log' mode='w' encoding='UTF-8'> disc: 0.187764, encoder : 1.581658, decoder : 56.367889


Generation of new samples

In [0]:
# Generate new samples

test_data = load_test()
assert test_data.shape == (num_of_test_instances, input_layer_size)

test_gen = buffered_gen(test, batch_n=aae.batch_size)
batch_x = test_gen.__next__()
assert len(batch_x[0]) == input_layer_size

enc = aae.sess.run(aae.encoded, feed_dict={aae.input_x: test_data})
assert enc.shape == (num_of_test_instances, latent_layer_size)

dec = aae.sess.run(aae.decoded, feed_dict={aae.input_x: batch_x})
assert dec.shape == (batch_size_for_model, output_layer_size)
assert len(dec[0]) == output_layer_size

def convert_to_fingerprint(arr):
  fp = []
  for a in arr:
    if (a < 0.5):
      fp.append(0)
    else:
      fp.append(1)
   
  return np.array(fp)

In [9]:
# Check one of the generated samples

generated_fingerprint = convert_to_fingerprint(dec[0])
generated_fingerprint

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0])

Analysis of generated samples

In [0]:
# Functions for analysis of generated samples

def calc_mean_mean(set):
  mean_mean = 0
  for a in set:
    mean_mean += np.mean(a)
  mean_mean /= len(set)
  
  return mean_mean

def calc_mean_var(set):
  mean_var = 0
  for a in set:
    mean_var += np.var(a)
  mean_var /= len(set)
  
  return mean_var

In [11]:
# Display analysis results

print ("mean mean = " + str(calc_mean_mean(dec)) + ", mean var = " + str(calc_mean_var(dec)))

mean mean = -2.2987413390073925, mean var = 8.140357475727797


Retrieving and removing of files

In [0]:
# Retrieve names of event files

x = subprocess.Popen(['ls', logdir], stdout=subprocess.PIPE)
(out, err) = x.communicate()
events = str(out.strip()).split("'")[1].split('\\n')

In [0]:
# Download necessary files

files.download('fpt.aae.95e.model.ckpt.data-00000-of-00001')
files.download('fpt.aae.95e.model.ckpt.index')
files.download('fpt.aae.95e.model.ckpt.meta')

files.download('fpt.aae.99e.model.ckpt.data-00000-of-00001')
files.download('fpt.aae.99e.model.ckpt.index')
files.download('fpt.aae.99e.model.ckpt.meta')

for event in events:
  files.download(logdir + event)

In [0]:
# Remove downloaded/unneeded files
# NOTE: Delete with caution

try:
  subprocess.call(['rm', 'fpt.aae.95e.model.ckpt.data-00000-of-00001'])
  subprocess.call(['rm', 'fpt.aae.95e.model.ckpt.index'])
  subprocess.call(['rm', 'fpt.aae.95e.model.ckpt.meta'])
  subprocess.call(['rm', 'fpt.aae.96e.model.ckpt.data-00000-of-00001'])
  subprocess.call(['rm', 'fpt.aae.96e.model.ckpt.index'])
  subprocess.call(['rm', 'fpt.aae.96e.model.ckpt.meta'])
  subprocess.call(['rm', 'fpt.aae.97e.model.ckpt.data-00000-of-00001'])
  subprocess.call(['rm', 'fpt.aae.97e.model.ckpt.index'])
  subprocess.call(['rm', 'fpt.aae.97e.model.ckpt.meta'])
  subprocess.call(['rm', 'fpt.aae.98e.model.ckpt.data-00000-of-00001'])
  subprocess.call(['rm', 'fpt.aae.98e.model.ckpt.index'])
  subprocess.call(['rm', 'fpt.aae.98e.model.ckpt.meta'])
  subprocess.call(['rm', 'fpt.aae.99e.model.ckpt.data-00000-of-00001'])
  subprocess.call(['rm', 'fpt.aae.99e.model.ckpt.index'])
  subprocess.call(['rm', 'fpt.aae.99e.model.ckpt.meta'])
  subprocess.call(['rm', 'fpt.aae.log'])
  subprocess.call(['rm', '-rvf', 'tmp'])
except:
  print ("Exception.")